In [1]:
import requests
import geopandas as gpd
import os
import zipfile
import base64
from ipyleaflet import Map, CircleMarker, basemaps, Polyline, Popup
from geopy import distance
from ipywidgets import Label, Button, Layout, Text, VBox, HTML
from IPython.display import display, HTML as HTMLDisplay
from shapely.geometry import LineString, Point, MultiPoint  # Import Point

m = Map(center=(0, 0), zoom=2, basemap=basemaps.OpenStreetMap.Mapnik)

label = HTML()

points = []
handler_state = [False]  # Use a list to hold the state

# Create instruction
instruction = HTML("<p>First choose a name for the hub, then select a representing point on the map. Continue with other points (hubs).</p>")
name_prompt = Text(value='', placeholder='Enter Hub Name', description='Hub Name:', disabled=False)

download_button = Button(description='Download SHP', layout=Layout(width='150px'), tooltip='Download Shapefile', disabled=True)
download_link = HTML("")

new_route_button = Button(description='New route', layout=Layout(width='150px'), tooltip='Create a new route')
undo_button = Button(description='Undo', layout=Layout(width='100px'), tooltip='Delete the previously created hub', disabled=True)
done_button = Button(description='Generate', layout=Layout(width='100px'), tooltip='Create a route', disabled=True)

line = None

def get_route(coordinates):
    formatted_coords = ';'.join([f'{coord[1]},{coord[0]}' for coord in coordinates])
    route_request = f'http://router.project-osrm.org/route/v1/driving/{formatted_coords}?overview=full&geometries=geojson'
    response = requests.get(route_request)
    data = response.json()
    route = data['routes'][0]['geometry']['coordinates']
    route = [(coord[1], coord[0]) for coord in route]
    return route

def handle_interaction(**kwargs):
    if handler_state[0] and kwargs.get('type') == 'click':
        coordinates = kwargs.get('coordinates')
        point_name = name_prompt.value

        marker = CircleMarker(location=coordinates, color="grey", radius=5, fill_opacity=1.0)
        m.add_layer(marker)

        label_html = HTML(value=f'<b>{point_name}</b>')
        popup = Popup(
            location=coordinates,
            child=label_html,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )
        m.add_layer(popup)

        points.append((coordinates, popup, marker, point_name))
        name_prompt.value = ''

def calculate_distance(b):
    global line
    if len(points) > 1:
        route = get_route([point[0] for point in points])
        if line in m.layers:
            m.remove_layer(line)
        line = Polyline(locations=route, color="blue", fill=False)
        m.add_layer(line)
        
        total_dist = 0
        dist_str = ""
        for i in range(1, len(points)):
            dist = distance.distance(points[i-1][0], points[i][0]).km
            total_dist += dist
            dist_str += "{} - {}: {:.2f} km<br>".format(points[i-1][3], points[i][3], dist)

        label.value = dist_str + '<b>Total distance: {:.2f} km</b>'.format(total_dist)

def handle_new_route_click(b):
    global line
    for point in points:
        m.remove_layer(point[1])  # Remove Popup
        m.remove_layer(point[2])  # Remove Marker
    points.clear()  # Clear Points
    if line in m.layers:  # Remove Polylines
        m.remove_layer(line)
    label.value = ''  # Clear Label
    handler_state[0] = True
    done_button.disabled = False
    undo_button.disabled = False
    download_link.value = ''  # Clear Download Link

def handle_done_button_click(b):
    handler_state[0] = False
    done_button.disabled = True
    undo_button.disabled = True
    download_button.disabled = False
    calculate_distance(b)

def handle_undo_button_click(b):
    if points:
        last_point = points.pop()
        m.remove_layer(last_point[1])  # Remove Popup
        m.remove_layer(last_point[2])  # Remove Marker
        if not points:
            undo_button.disabled = True

def handle_download_button_click(b):
    # Create a Shapefile
    route = get_route([point[0] for point in points])
    # Swap latitude and longitude
    route = [(coord[1], coord[0]) for coord in route]
    gdf_route = gpd.GeoDataFrame(geometry=[LineString(route)], crs="EPSG:4326")
    gdf_route.to_file("route.shp")

    # Create a Shapefile for the points
    all_point_names = [point[3] if point[3] else 'route_point' for point in points] + ['route_point'] * len(route)
    all_point_geometries = [Point(point[0][1], point[0][0]) for point in points] + [Point(coord) for coord in route]
    all_point_lats = [point[0][0] for point in points] + [coord[0] for coord in route]
    all_point_longs = [point[0][1] for point in points] + [coord[1] for coord in route]

    gdf_points = gpd.GeoDataFrame({'Name': all_point_names, 'Latitude': all_point_lats, 'Longitude': all_point_longs,
                                    'geometry': all_point_geometries}, crs="EPSG:4326")
    gdf_points.to_file("points.shp")

    # Create a Shapefile for the segments
    segments = [LineString([route[i], route[i + 1]]) for i in range(len(route) - 1)]
    segment_names_start = [points[i][3] for i in range(len(points) - 1)] + ['route_point'] * (len(route) - len(points))
    segment_names_end = [points[i + 1][3] for i in range(len(points) - 1)] + ['route_point'] * (len(route) - len(points))
    segment_lats_start = [points[i][0][0] for i in range(len(points) - 1)] + [route[i][0] for i in range(len(route) - len(points))]
    segment_longs_start = [points[i][0][1] for i in range(len(points) - 1)] + [route[i][1] for i in range(len(route) - len(points))]
    segment_lats_end = [points[i + 1][0][0] for i in range(len(points) - 1)] + [route[i + 1][0] for i in range(len(route) - len(points))]
    segment_longs_end = [points[i + 1][0][1] for i in range(len(points) - 1)] + [route[i + 1][1] for i in range(len(route) - len(points))]

    gdf_segments = gpd.GeoDataFrame({'Start': segment_names_start, 'Lat_Start': segment_lats_start, 'Long_Start': segment_longs_start,
                                     'End': segment_names_end, 'Lat_End': segment_lats_end, 'Long_End': segment_longs_end,
                                     'geometry': segments}, crs="EPSG:4326")
    gdf_segments.to_file("route_parts.shp")

    # Zip the Shapefiles
    with zipfile.ZipFile('route.zip', 'w') as zipf:
        for file in os.listdir():
            if (file.startswith('route.') or file.startswith('points.') or file.startswith('route_parts.')) and not file.endswith('.zip'):
                zipf.write(file)
                os.remove(file)

    # Create download link
    download_link.value = "<a href='route.zip' download>Click here to download: route.zip</a>"

new_route_button.on_click(handle_new_route_click)
done_button.on_click(handle_done_button_click)
undo_button.on_click(handle_undo_button_click)
download_button.on_click(handle_download_button_click)

m.on_interaction(handle_interaction)

display(VBox([instruction, new_route_button, undo_button, done_button, download_button, download_link, name_prompt]))
display(m)
display(label)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

HTML(value='')